In [ ]:
!pip3 install openai

In [ ]:
import csv
import os

from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
client = OpenAI()

# Pull in data

[Link](https://drive.google.com/file/d/1svDBMaYwv1uh_Z5yN9ZZl2n60Ep56fO0/view?usp=sharing) to download google_deepmind.txt

[Link](https://drive.google.com/file/d/1wVHzI8N3oCyciC-tHERaUasU6OmFjxbC/view?usp=sharing) to download response_pairs.csv

In [ ]:
# Pull in context data
context = ""
with open("google_deepmind.txt", "r") as file:
    context = file.read()

# Pull in response pair data
response_pairs = []
with open("response_pairs.csv", "r") as file:
  response_pairs = list(csv.reader(file, delimiter=","))

questions = [row[0] for row in response_pairs]
answers = [row[1] for row in response_pairs]

# Define prompt templates

In [ ]:
prompt_template1 = "Context: {context} Question: {question} Answer the question concisely based on information provided in the context. The answer should completely and accurately address the question asked using relevant context. Answer:"

prompt_template2 = "{context} {question}"

# Prompt function

In [ ]:
def prompt(prompt_template):
  sampled_answers = []

  for question in questions:
    filled_prompt_template = prompt_template.format(context=context, question=question)
    response_text = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": filled_prompt_template},
        ]
    ).json()

    response_obj = json.loads(response_text)
    content = response_obj["choices"][0]["message"]["content"]
    sampled_answers.append(content)

  return sampled_answers

# Define evaluation template

In [ ]:
evaluation_template = "You are given a correct answer and a sampled answer. You must determine whether the sampled answer is correct based on the correct answer. Here is the data:\n[BEGIN DATA]\n************\n[question]: {question}\n************\n[correct answer]: {correct_answer}\n************\n[sampled_answer]: {sampled_answer}\n[END DATA]\n\nYour response must be a single number, either 1 or 0, and should not contain any text or characters aside from that word. 1 means that the sampled answer answers the question as well as the given correct answer. 0 means it does not."


# Eval function

In [ ]:
def eval(sampled_answers):
  total = 0

  for i, sampled_answer in enumerate(sampled_answers):
    filled_evaluation_template = evaluation_template.format(question=questions[i], correct_answer=answers[i], sampled_answer=sampled_answer)
    response_text = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": filled_evaluation_template},
        ]
    ).json()

    response_obj = json.loads(response_text)
    content = response_obj["choices"][0]["message"]["content"]
    total += int(content)

  score = total/len(sampled_answers)
  return score


# Prompt model using template #1

In [ ]:
template1_answers = prompt(prompt_template1)


# Evaluate template #1 answers

In [ ]:
score1 = eval(template1_answers)

In [ ]:
print(score1)

# Prompt model with template #2

In [ ]:
template2_answers = prompt(prompt_template2)

# Evaluate template #2 answers

In [ ]:
score2 = eval(template2_answers)

In [ ]:
print(score2)